##코로나 관련 트윗 분석
1. 2020년 5월14일-16일 동안 생성된 '코로나19' 키워드를 가진 트윗 10,000개 수집(크롤링)
2. 전처리 (불용어 제거)
3. FastText로 워드임베딩 생성 (skipgram, minCount=10)
4. (ongoing) 추가로 Term Frequency-Inverse Document Frequency (TF-IDF) 기반으로 Word2Vec 구현
5. Nearest neigbors (관련어) 출력
6. (ongoing) 벡터공간 시각화

In [1]:
# GetOldTweet3 라이브러리로 트윗 크롤링
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#1. 트윗 크롤링
#2020년5월14-16일간 생성된 '코로나19' 키워드를 가진 트윗을 크롤링
#맥스트윗은 10,000개로 제한. HTTP 429 “Too Many Requests” 에러 방지
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('코로나19')\
                                           .setSince("2020-05-14")\
                                           .setUntil("2020-05-19")\
                                           .setMaxTweets(10000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

tweets=[]
for cnt, tw in enumerate(tweet):
  tweets.append(tw.text)
  if cnt%100==0:
    print(tw.text)
  
print("total %d tweets compiled" %len(tweets))

ㅇ 적용시기 : 2020.05.24(일) * 코로나19로 인한 한시적 조치이며, 추후 상황에 따라 재조정 할 예정입니다. ㅇ 현행 : 22,000원 → 변경 후 : 22,690원 ☞ http://overseas.mofa.go.kr/jp-osaka-ko/brd/m_20228/view.do?seq=1295929&amp;srchFr=&amp;srchTo=&amp;srchWord=&amp;srchTp=&amp;multi_itm_seq=0&amp;itm_seq_1=0&amp;itm_seq_2=0&amp;company_cd=&amp;company_nm=&amp;page=1
[블루오션스탁]모더나, 코로나19 백신 임상시험 고무적인 결과...국내 파미셀 주목 
[천지일보 시론] 코로나19와 방역당국에 대해 신천지교회가 바라보는 ‘특별한 시선(視線)’ 
미 바이오기업 모더나 "코로나19 백신 임상시험서 항체 형성"(종합) | 다음 뉴스 
코로나19에 대하여 - 뜻, 증상, 예방법 그리고 확산 원인 https://loveismeet.tistory.com/672 #코로나19 #코로나뜻 #코로나증상
2020.5.18.(월) 오스트리아와 슬로베니아의 코로나19 발생 현황 및 오스트리아 국경통제 상황 안내 http://overseas.mofa.go.kr/at-ko/brd/m_8461/view.do?seq=1347070&amp;srchFr=&amp;srchTo=&amp;srchWord=&amp;srchTp=&amp;multi_itm_seq=0&amp;itm_seq_1=0&amp;itm_seq_2=0&amp;company_cd=&amp;company_nm=&amp;page=1 https://www.facebook.com/KoreanEmbassyAustria.kr/posts/3122304081165522?__xts__%5B0%5D=68.ARAKKf-naza98RkdtcsP1uUrCwwRiSJWn8hNjdxKKHMJ_RxDPllRyNYj4oytIusWYR2qNsSM4ivSOVz8iZcMZ1s

In [4]:
#df=pd.read_csv("output_got.csv")
#tweet.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/tweet_crawling/tweet_corpus.csv', encoding='utf-8-sig')
#크롤링한 GetOldTweets3 모델 타입을 처리가능한 리스트로 변환
total=[]
for tw in tweet:
  total.append(tw.text)
print(total[0])

ㅇ 적용시기 : 2020.05.24(일) * 코로나19로 인한 한시적 조치이며, 추후 상황에 따라 재조정 할 예정입니다. ㅇ 현행 : 22,000원 → 변경 후 : 22,690원 ☞ http://overseas.mofa.go.kr/jp-osaka-ko/brd/m_20228/view.do?seq=1295929&amp;srchFr=&amp;srchTo=&amp;srchWord=&amp;srchTp=&amp;multi_itm_seq=0&amp;itm_seq_1=0&amp;itm_seq_2=0&amp;company_cd=&amp;company_nm=&amp;page=1


In [0]:
import pandas as pd

#원본 Corpus를 CSV파일로 저장
df = pd.DataFrame(total)
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/4.tweet_analytics/data/output_twcorpus.csv', encoding='utf-8-sig', index=False)

In [6]:
#2. 한국어 전처리 (불용어제거) - 코엔엘파이 라이브러리 사용
!pip3 install konlpy    # Python 3.x

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 3.6MB 44.8MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [7]:
#불용어 제거
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()
pprint(kkma.nouns(total[3]))
for i in range(len(total)):
  total[i]=" ".join(kkma.nouns(total[i]))

print(total[3])

['힘', '타사', '코로나', '코로나19', '19', '타격', '다음', '뉴스']
힘 타사 코로나 코로나19 19 타격 다음 뉴스


In [0]:
#전처리된 Corpus를 CSV파일로 저장
df = pd.DataFrame(total)
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/4.tweet_analytics/data/output_twcorpus_pp.csv', encoding='utf-8-sig', index=False)

In [9]:
#3. Fasttext로 워드임베딩 생성 (model='skipgram', minCount=10)
!pip install fasttext
import fasttext

# Skipgram model :
model = fasttext.train_unsupervised('/content/drive/My Drive/Colab Notebooks/NLP/4.tweet_analytics/data/output_twcorpus_pp.csv', model='skipgram', minCount=15)

# or, cbow model :
#model = fasttext.train_unsupervised('', model='cbow')

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3018239 sha256=7fd924082490f678ec3873fb7e3bbce3c8f1602c8b75445027e0ccf46d657782
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [10]:
print(model.words)   # list of words in dictionary
print(model['코로나19']) # get the vector of the word '코로나19'

['</s>', '코로나', '19', '코로나19', '뉴스', '신천지', '일', '명', '1', '5', '진자', '이태원', '2', '출처', '시', '월', '교회', '수', '3', '감염', '중', '방역', '클럽', '다음', '진', '네이버', '관련', '4', '만', '확산', '등', '발생', '전', '검사', '가짜', '6', '중국', '대응', '극복', '가짜뉴스', '사태', '신종', '예수', '한국', '세계', '사람', '방문', '차', '마스크', '체크', '대', '우리', '바이러스', '예방', '위', '지원', '팩트', '미국', '바', '5월', '0', '현황', '8', '18', '팩트체크', '신천지예수교회', '거리', '이', '분', '9', '7', '발', '서울', '15', '년', '구', '속', '지역', '데', '내', '번', '백신', '정부', '10', '나', '이후', '경제', '신', '로그', '집단', '국내', '환자', '신규', '다음뉴스', '방', '음성', '격리', '지원금', '원', '대구', '추가', '생활', '상황', '안전', '위기', '기', '긴급', '공유', '백', '경기', '2020', '종합', '종', '20', '국민', '일보', '재난', '거', '이러스', '문', '후', '사실', '미', '기준', '확인', '개', '때', '일본', '도', '감사', '코로나바이러스', '때문', '온라인', '16', '당국', '집', '의료진', '사망자', '김', '기자', '판정', '말', '오늘', '감염증', '안', '30', '님', '방법', '건강', '최대', '17', '공식', '사회적', '사망', '대책', '정국', '조사', '사회', '선', '해외', '특별', '강제', '학교', '천', '생각', '본부', '자가', '역', '예배', '고'

In [23]:
model.get_nearest_neighbors('한국')

[(0.7550534009933472, '한국경제'),
 (0.7296176552772522, '경제'),
 (0.7095469236373901, '독일'),
 (0.6939200162887573, '사무총장'),
 (0.6855972409248352, '각국'),
 (0.6828752160072327, '발언'),
 (0.6758472323417664, '정치'),
 (0.6744129061698914, '일본'),
 (0.6564424633979797, '한'),
 (0.6520951390266418, '총회')]

In [24]:
!python3 -m pip install -U pygame==2.0.0.dev6 --user
#!apt-get install python-pygame
!pip install git+https://github.com/e9t/PyTagCloud.git

     |████████████████████████████████| 15.2MB 310kB/s 
  Cloning https://github.com/e9t/PyTagCloud.git to /tmp/pip-req-build-n1anlx0t
  Running command git clone -q https://github.com/e9t/PyTagCloud.git /tmp/pip-req-build-n1anlx0t
  Created wheel for pytagcloud: filename=pytagcloud-0.3.5-cp36-none-any.whl size=16467702 sha256=10a1e5faa86f6f706dbc59aa0122c9d07f472eaacb9900beaff956a9a7eba968
  Stored in directory: /tmp/pip-ephem-wheel-cache-ic6jjjao/wheels/7c/dd/2a/1b943312df4faec266e7e02a767ecbc246f3cd19d37c45e4c6
Successfully built pytagcloud


In [26]:
#! /usr/bin/python2.7
# -*- coding: utf-8 -*-
from collections import Counter
import urllib
import random
import webbrowser

from konlpy.tag import Hannanum
from lxml import html
import pytagcloud # requires Korean font support
import sys

if sys.version_info[0] >= 3:
    urlopen = urllib.request.urlopen
else:
    urlopen = urllib.urlopen


r = lambda: random.randint(0,255)
color = lambda: (r(), r(), r())

def get_corona_text(cor_text):
    url = 'http://pokr.kr/bill/%s/text' % cor_text
    response = urlopen(url).read().decode('utf-8')
    page = html.fromstring(response)
    text = page.xpath(".//div[@id='bill-sections']/pre/text()")[0]
    return text

def get_tags(text, ntags=50, multiplier=10):
    h = Hannanum()
    nouns = h.nouns(text)
    count = Counter(nouns)
    return [{ 'color': color(), 'tag': n, 'size': c*multiplier }\
                for n, c in count.most_common(ntags)]

def draw_cloud(tags, filename, fontname='Noto Sans CJK', size=(800, 600)):
    pytagcloud.create_tag_image(tags, filename, fontname=fontname, size=size)
    webbrowser.open(filename)


cor_text = '한국'
text = get_corona_text(cor_text)
tags = get_tags(text)
print(tags)
draw_cloud(tags, 'wordcloud.png')

ModuleNotFoundError: ignored

In [0]:
model.get_word_id('한국')

47

In [0]:
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.words:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [0]:
tsne_plot(model)

NameError: ignored